In [1]:
import os 
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
data_dir='../data/'

In [2]:
train=pd.read_csv(data_dir+'train_features.csv')
train_labels=pd.read_csv(data_dir+'train_labels.csv')
test=pd.read_csv(data_dir+'test_features.csv')
submission=pd.read_csv(data_dir+'sample_submission.csv')

In [3]:
len(test.id.unique())

782

In [4]:
train_labels

,id,label,label_desc
0,0,37,Shoulder Press (dumbbell)
1,1,26,Non-Exercise
2,2,3,Biceps Curl (band)
3,3,26,Non-Exercise
4,4,26,Non-Exercise
...,...,...,...
3120,3120,26,Non-Exercise
3121,3121,26,Non-Exercise
3122,3122,15,Dynamic Stretch (at your own pace)
3123,3123,26,Non-Exercise


In [5]:
train[train['id']==0].iloc[:,1:]

,time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z
0,0,1.206087,-0.179371,-0.148447,-0.591608,-30.549010,-31.676112
1,1,1.287696,-0.198974,-0.182444,0.303100,-39.139103,-24.927216
2,2,1.304609,-0.195114,-0.253382,-3.617278,-44.122565,-25.019629
3,3,1.293095,-0.230366,-0.215210,2.712986,-53.597843,-27.454013
4,4,1.300887,-0.187757,-0.222523,4.286707,-57.906561,-27.961234
...,...,...,...,...,...,...,...
595,595,0.985242,-0.326122,-0.354528,-14.903280,20.172339,22.973018
596,596,1.052837,-0.220710,-0.413472,-10.857025,19.786856,23.174597
597,597,1.025643,-0.227845,-0.354516,-2.334243,25.768654,18.932070
598,598,1.031553,-0.387862,-0.277857,-9.710746,28.697694,20.631577


In [6]:
submission

,id,0,1,2,3,4,5,6,7,8,...,51,52,53,54,55,56,57,58,59,60
0,3125,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,3126,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3127,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3128,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,3129,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
777,3902,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
778,3903,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
779,3904,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
780,3905,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
test['id'][0]-1

3124

In [8]:
class HealthDataset(Dataset):
    def __init__(self,feature_df,label=None,train_mode=True):
        self.X = feature_df
        self.train_mode= train_mode
        if self.train_mode:
            self.y = label
        else:
            self.first_test_id = self.X['id'][0]
        
    def __len__(self):
        return len(self.X.id.unique())
    
    def __getitem__(self,idx):
        if self.train_mode:
            return (np.array(self.X[self.X['id']==idx].iloc[:,1:]),np.array(self.y[self.y['id']==idx]['label']))
        else:
            return np.array(self.X[self.X['id']==idx+self.first_test_id].iloc[:,1:])

In [9]:
class LSTMNet(nn.Module):
    def __init__(self,device,input_size,hidden_size=30,num_layers=1,output_size=61):
        super(LSTMNet, self).__init__()
        self.device=device
        
        self.input_size = input_size
        self.hidden_size= hidden_size
        self.num_layers = num_layers
        
        self.dropout = nn.Dropout(0.6)
        self.softmax = nn.Softmax
        
        self.lstm= nn.LSTM(input_size,hidden_size,num_layers)
        self.fc1 = nn.Linear(hidden_size,hidden_size)
        self.fc2 = nn.Linear(hidden_size,output_size)
        
    def init_hidden_and_cell(self, batch_size):
        h0 = torch.zeros(self.num_layers,batch_size, self.hidden_size, requires_grad=True).to(self.device)
        c0 = torch.zeros(self.num_layers,batch_size, self.hidden_size, requires_grad=True).to(self.device)
        return h0, c0
        
    def forward(self,X,batch_size):
        h0,c0 = self.init_hidden_and_cell(batch_size) 
        out,_ = self.lstm(X,(h0,c0))
        print(out)
        out = out[-1,:,:]
        out = self.fc1(out)
        out = self.dropout(out)
        out = self.fc2(out)
        return out
        
    

In [10]:
train_dataset = HealthDataset(train,train_labels)
test_dataset = HealthDataset(test,train_mode=False)

In [11]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_epochs = 10
batch_size = 120
learning_rate = 0.001

input_size = 7
sequence_length=600

In [12]:
device

device(type='cpu')

In [21]:
model = LSTMNet(device,input_size)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 



In [38]:
# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           drop_last=True,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=1, 
                                          shuffle=False)

In [23]:
# Train the model
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (X, labels) in enumerate(train_loader):  
        X = X.float().reshape(sequence_length,batch_size,-1).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(X,batch_size)
        loss = criterion(outputs, labels.view(-1))
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 5 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')


Epoch [1/10], Step [5/26], Loss: 4.1015


KeyboardInterrupt: 

In [64]:
submission.iloc[1,1:]

0     0
1     0
2     0
3     0
4     0
     ..
56    0
57    0
58    0
59    0
60    0
Name: 1, Length: 61, dtype: int64

In [62]:
outputs.cpu().detach().numpy()[0]

array([ 0.04688787,  0.22598483, -0.15428557,  0.22852793, -0.08554859,
       -0.14721918, -0.16928847,  0.03377847, -0.17818247,  0.33163273,
        0.04120053, -0.39764783,  0.08577716,  0.08304121,  0.06710932,
       -0.05068767, -0.13037862, -0.05046587,  0.14745343, -0.00298305,
        0.11244708, -0.02229813,  0.05343029, -0.10128136, -0.02691942,
       -0.01969356,  0.01459435,  0.02405441, -0.28091633, -0.1058386 ,
       -0.1964784 ,  0.13919574,  0.50546986, -0.02370975, -0.12723687,
        0.02722047,  0.09158865, -0.02846263, -0.13477308,  0.08473416,
        0.1484694 , -0.2535581 , -0.15922949, -0.09316327, -0.03865248,
       -0.09983855,  0.03773701, -0.01287023, -0.23663607,  0.01803932,
       -0.01314484,  0.1494862 ,  0.08561678, -0.4165019 , -0.3165934 ,
       -0.192538  , -0.09987211,  0.16285391, -0.11543726,  0.58592296,
       -0.31860226], dtype=float32)

In [63]:
len(submission.iloc[i,:])

62

In [61]:
submission.iloc[i,:]= outputs.cpu().detach().numpy()[0]

ValueError: Must have equal len keys and value when setting with an iterable

In [65]:
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    for i, X in enumerate(test_loader):
        X = X.float().reshape(sequence_length,1,-1).to(device)
        outputs = model(X,1)
        submission.iloc[i,1:]= outputs.cpu().detach().numpy()[0]
        _, predicted = torch.max(outputs.data, 1)


torch.Size([600, 1, 7])
tensor([[-0.2522,  0.2934,  0.1451, -0.0265,  0.2151, -0.0629, -0.1170, -0.2269,
         -0.2438, -0.0731,  0.0780, -0.7241,  0.3584,  0.3122,  0.0653, -0.2027,
         -0.1357, -0.1071,  0.2490,  0.0698,  0.2147, -0.0843,  0.1877, -0.1233,
         -0.0263, -0.0892,  0.1355, -0.0696, -0.2335,  0.0885, -0.4644,  0.3936,
          0.6168, -0.1127, -0.1218,  0.1758, -0.3231, -0.1760, -0.0658, -0.2835,
         -0.0189,  0.0765,  0.0686, -0.2857,  0.1331, -0.2069,  0.1116, -0.0037,
         -0.4280, -0.0185, -0.0011,  0.2843,  0.2680, -0.6336, -0.3387, -0.2492,
          0.0417, -0.0422, -0.0121,  0.4069, -0.4116]])
       id         0        1         2         3        4         5         6  \
0    3125 -0.252193  0.29337  0.145122 -0.026517  0.21505 -0.062923 -0.116975   
1    3126  0.000000  0.00000  0.000000  0.000000  0.00000  0.000000  0.000000   
2    3127  0.000000  0.00000  0.000000  0.000000  0.00000  0.000000  0.000000   
3    3128  0.000000  0.00000 